In [1]:
%load_ext autoreload
%autoreload 2

from theano.sandbox import cuda
cuda.use('gpu1')

import os, sys
from luvgg16 import VGG16
import numpy as np
from keras.layers import Dense, Convolution2D

NB_HOME = os.path.dirname(os.getcwd())
DATA_HOME = NB_HOME + '/data/redux'

Using gpu device 1: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)
/home/lukaka/anaconda2/envs/py3/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
print(NB_HOME)
print(DATA_HOME)

/home/lukaka/Desktop/lab/dog cat (kaggle)
/home/lukaka/Desktop/lab/dog cat (kaggle)/data/redux


### paly with sample data

In [3]:
vgg = VGG16()
vgg.fine_tune()
vgg.compile()

sample_train = vgg.get_batch_generator(path=DATA_HOME+'/sample/train')
sample_valid = vgg.get_batch_generator(path=DATA_HOME+'/sample/validation')

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [4]:
vgg.fit(sample_train, sample_valid)

Epoch 1/1
2s - loss: 0.6777 - acc: 0.8600 - val_loss: 0.0094 - val_acc: 1.0000


### real data
plan:
1. overfit the model first by removing regularization (dropout)
2. start regularize the model by:
  * using dropout
  * data augmentation
  * use batchnorm
    * since the orignal VGG weights, and in theory you should insert BN layer everywhere, JH ends up retrain VGG with BN with original imagenet data..

In [5]:
vgg = VGG16()
vgg.fine_tune()


# dense_layers = [layer for layer in vgg.model.layers if isinstance(layer, Dense)]

# for layer in dense_layers[-2:]:
#     layer.trainable = True

# vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_15 (Convolution2D) (None, 64, 224, 224)  36928       convolution2d_14[0][0]           
____________________________________________________________________________________________________
maxpooling2d_6 (MaxPooling2D)    (None, 64, 112, 112)  0           convolution2d_15[0][0]           
___________________________________________________________________________________________

In [6]:
vgg.compile(lr=1e-3)

train = vgg.get_batch_generator(path=DATA_HOME+'/train', augmentation=False)
valid = vgg.get_batch_generator(path=DATA_HOME+'/validation')

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [7]:
vgg.fit(train, valid, nb_epoch=2)

Epoch 1/2
229s - loss: 0.0801 - acc: 0.9778 - val_loss: 0.1071 - val_acc: 0.9768
Epoch 2/2
227s - loss: 0.0593 - acc: 0.9845 - val_loss: 0.1096 - val_acc: 0.9788


as you can see, the val loss is not decresing. More epoch will not help.

### save model

In [29]:
MODEL_WEIGHT_PATH = DATA_HOME+'/result/vgg_model.h5'

In [30]:
vgg.save_weight(MODEL_WEIGHT_PATH)